In [2]:
import pandas as pd

In [3]:
df_basic_info = pd.read_csv("../data/processed/stocks-basic-info.csv")
df_history = pd.read_csv("../data/processed/stocks-history.csv")
df_fundaments = pd.read_csv("../data/processed/stocks-fundaments.csv")
df_right_prices = pd.read_csv("../data/processed/stocks-right-prices.csv")

In [4]:
def get_sectors():
    return df_basic_info[["SETOR", "SUBSETOR", "SEGMENTO"]].drop_duplicates()

In [13]:
def get_main_ticker(tickers):
    tickers = tickers.split(";")

    for ticker in tickers:
        if ticker[4] == "4":
            return ticker

    for ticker in tickers:
        if ticker[4] == "3":
            return ticker

    return tickers[0]

In [24]:
def get_companies_by_segmento(segmento):
    df_tmp = df_basic_info[df_basic_info["SEGMENTO"] == segmento].copy()
    df_tmp["MAIN_TICKER"] = df_tmp["TICKERS"].apply(get_main_ticker)
    return df_tmp[["CD_CVM", "DENOM_COMERC", "MAIN_TICKER"]]

In [50]:
def get_latest_values_by_ticker(ticker):
    df_history_tmp = df_history[df_history["TICKER"] == ticker]
    cd_cvm = df_history_tmp.iloc[0]["CD_CVM"]

    df_fundaments_tmp = df_fundaments[df_fundaments["CD_CVM"] == cd_cvm]
    last_dt_fim_exerc = df_fundaments_tmp["DT_FIM_EXERC"].max()

    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["DT_FIM_EXERC"] == last_dt_fim_exerc]
    df_fundaments_tmp_2 = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] == -1]
    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] != -1]
    df_fundaments_tmp = df_fundaments_tmp.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA_ROLLING_YEAR").reset_index()
    df_fundaments_tmp_2 = df_fundaments_tmp_2.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA").reset_index()

    df_fundaments_tmp = pd.concat([df_fundaments_tmp, df_fundaments_tmp_2], axis=1)
    df_fundaments_tmp = df_fundaments_tmp.drop("DT_FIM_EXERC", axis=1)

    df_history_tmp = df_history_tmp.tail(1).reset_index(drop=True)

    df_right_prices_tmp = df_right_prices[df_right_prices["TICKER"] == ticker]
    df_right_prices_tmp = df_right_prices_tmp.drop(["CD_CVM", "TICKER"], axis=1).reset_index(drop=True)
    
    df_latest_values = pd.concat([df_history_tmp, df_right_prices_tmp, df_fundaments_tmp], axis=1)

    total_stocks = df_basic_info[df_basic_info["CD_CVM"] == cd_cvm].iloc[0]["NUM_TOTAL"]

    df_latest_values["MARKET_CAP"] = df_latest_values["PRICE"] * total_stocks

    return df_latest_values

In [58]:
def get_latest_values_by_segmento(segmento):
    df_segment = pd.DataFrame()

    for ticker in get_companies_by_segmento(segmento=segmento)["MAIN_TICKER"].values:
        df_segment = pd.concat([df_segment, get_latest_values_by_ticker(ticker=ticker)])
        
    return df_segment.sort_values(by="MARKET_CAP", ascending=False)

In [10]:
get_sectors()

,SETOR,SUBSETOR,SEGMENTO
0,Financeiro,Intermediários Financeiros,Bancos
5,Utilidade Pública,Energia Elétrica,Energia Elétrica
7,Utilidade Pública,Água e Saneamento,Água e Saneamento


In [25]:
get_companies_by_segmento(segmento="Bancos")

,CD_CVM,DENOM_COMERC,MAIN_TICKER
0,906,BANCO BRADESCO S.A.,BBDC4
1,22616,BANCO UBS PACTUAL S/A,BPAC3
2,922,BANCO DA AMAZONIA S.A.,BAZA3
3,1023,BANCO DO BRASIL S.A.,BBAS3
4,20532,BANCO SANTANDER (BRASIL) S.A.,SANB4
14,19348,ITAÚ UNIBANCO,ITUB4


In [12]:
get_latest_values_by_ticker(ticker="BBAS3")

,DATE,CD_CVM,TICKER,PRICE,PL,DIVIDEND_YIELD,DIVIDEND_PAYOUT,PVP,BAZIN,EBIT,PROFIT,CGAR_5_YEARS,EQUITY,ROE
0,2024-10-21,1023,BBAS3,26.559999,4.35535,0.094818,0.412965,0.829582,38.205378,4.108161e+10,3.494804e+10,0.150059,1.834792e+11,0.190474


In [86]:
df_tmp = get_latest_values_by_segmento(segmento="Bancos")
df_segment_tmp = df_tmp.drop(["DATE", "CD_CVM", "TICKER", "BAZIN", "PRICE"], axis=1).reset_index()

market_cap_kpis = ["EBIT", "PROFIT", "EQUITY", "MARKET_CAP"]
market_cap_total = df_segment_tmp["MARKET_CAP"].sum()

df_segment_tmp.drop(market_cap_kpis, axis=1).mean().to_frame().T

df_segment_tmp[market_cap_kpis].apply(lambda x:  df_segment_tmp.iloc[x.index]["MARKET_CAP"] / market_cap_total)#.sum().to_frame().T

,EBIT,PROFIT,EQUITY,MARKET_CAP
0,0.361823,0.361823,0.361823,0.361823
1,0.183875,0.183875,0.183875,0.183875
2,0.171863,0.171863,0.171863,0.171863
3,0.160450,0.160450,0.160450,0.160450
4,0.116669,0.116669,0.116669,0.116669
5,0.005320,0.005320,0.005320,0.005320


In [83]:
df_segment_tmp

,index,PL,DIVIDEND_YIELD,DIVIDEND_PAYOUT,PVP,EBIT,PROFIT,CGAR_5_YEARS,EQUITY,ROE,MARKET_CAP
0,0,9.074735,0.067122,0.609115,1.693078,4.502400e+10,3.782400e+10,0.071490,2.027330e+11,0.186571,3.432428e+11
1,0,15.934074,0.017279,0.275318,3.286497,1.197988e+10,1.094716e+10,0.230281,5.307558e+10,0.206256,1.744328e+11
2,0,12.216371,0.097275,1.188344,0.987205,8.911096e+09,1.334586e+10,-0.074140,1.651512e+11,0.080810,1.630380e+11
3,0,4.355350,0.094818,0.412965,0.829582,4.108161e+10,3.494804e+10,0.150059,1.834792e+11,0.190474,1.522109e+11
4,0,10.480266,0.042861,0.449198,0.941354,1.597751e+10,1.056064e+10,-0.057066,1.175735e+11,0.089822,1.106783e+11
5,0,3.827728,0.073597,0.281708,0.781008,1.618939e+09,1.318372e+09,0.466331,6.461355e+09,0.204040,5.046369e+09
